In [6]:
import cv2
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the training dataset
data_train_path = 'data/train/DataTrain.csv'
image_folder_train = 'data/train'
train_data = pd.read_csv(data_train_path, sep=';')
train_data['ImagePath'] = train_data['NameofFile'].apply(lambda x: os.path.join(image_folder_train, x))

# Function to preprocess images
def preprocess_images(image_paths):
    images = []
    for image_path in image_paths:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (128, 64))
        image = image / 255.0
        images.append(image)
    return np.array(images)

# Preprocess images
X = preprocess_images(train_data['ImagePath'])

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_data['Vehicleregistrationplate'])

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape images to include channel dimension
X_train = X_train.reshape(X_train.shape[0], 64, 128, 1)
X_val = X_val.reshape(X_val.shape[0], 64, 128, 1)

# Build the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')



Epoch 1/20
20/20 [==============================] - 3s 124ms/step - loss: 6.4245 - accuracy: 0.0016 - val_loss: 6.4243 - val_accuracy: 0.0000e+00
Epoch 2/20
20/20 [==============================] - 2s 116ms/step - loss: 6.4095 - accuracy: 0.0031 - val_loss: 6.4966 - val_accuracy: 0.0000e+00
Epoch 3/20
20/20 [==============================] - 2s 117ms/step - loss: 6.3504 - accuracy: 0.0063 - val_loss: 6.5423 - val_accuracy: 0.0000e+00
Epoch 4/20
20/20 [==============================] - 2s 124ms/step - loss: 6.2291 - accuracy: 0.0219 - val_loss: 6.8767 - val_accuracy: 0.0000e+00
Epoch 5/20
20/20 [==============================] - 2s 120ms/step - loss: 5.8280 - accuracy: 0.0625 - val_loss: 7.4515 - val_accuracy: 0.0000e+00
Epoch 6/20
20/20 [==============================] - 2s 123ms/step - loss: 4.9915 - accuracy: 0.1234 - val_loss: 8.8630 - val_accuracy: 0.0250
Epoch 7/20
20/20 [==============================] - 2s 123ms/step - loss: 3.6598 - accuracy: 0.3203 - val_loss: 13.0741 - val_ac

In [7]:
model.save('car_plate_number_model.h5')
print('Model saved successfully.')

# Load the test dataset
data_test_path = 'data/test/DataTest.csv'
image_folder_test = 'data/test'
test_data = pd.read_csv(data_test_path, sep=';')
test_data['ImagePath'] = test_data['Name of File'].apply(lambda x: os.path.join(image_folder_test, x))

# Preprocess test images
X_test = preprocess_images(test_data['ImagePath'])
X_test = X_test.reshape(X_test.shape[0], 64, 128, 1)

# Predict car plate numbers for the test dataset
predictions = model.predict(X_test)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

# Save the predictions to a CSV file
predictions_df = pd.DataFrame({'Image': test_data['Name of File'], 'Predicted Plate Number': predicted_labels})
predictions_df.to_csv('predictions.csv', index=False)

Model saved successfully.
4/4 [==============================] - 0s 21ms/step
